# Plots

In [209]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import json
from plotly.subplots import make_subplots

In [170]:
df = pd.read_csv('../data/processed/squawk7700_processed_final_v2.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 635 entries, 0 to 634
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   flight_id       635 non-null    object 
 1   icao24          635 non-null    object 
 2   registration    635 non-null    object 
 3   typecode        635 non-null    object 
 4   origin          623 non-null    object 
 5   landing         447 non-null    object 
 6   destination     621 non-null    object 
 7   diverted        219 non-null    object 
 8   tweet_problem   301 non-null    object 
 9   tweet_result    301 non-null    object 
 10  tweet_fueldump  301 non-null    object 
 11  avh_id          84 non-null     object 
 12  avh_problem     84 non-null     object 
 13  avh_result      84 non-null     object 
 14  avh_fueldump    84 non-null     object 
 15  airlineName     635 non-null    object 
 16  isFreighter     635 non-null    bool   
 17  productionLine  635 non-null    obj

## Age BAR

##### Data prep

In [171]:
bins = np.arange(0, df['ageYears'].max() + 5, 5)
df['age_categories'] = pd.cut(df['ageYears'], bins=bins, right=False)

In [172]:
df.head()

,flight_id,icao24,registration,typecode,origin,landing,destination,diverted,tweet_problem,tweet_result,...,avh_id,avh_problem,avh_result,avh_fueldump,airlineName,isFreighter,productionLine,ageYears,manufacturer,age_categories
0,ARG1511_20180101,e06442,LV-FQB,B738,SACO,SABE,SABE,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Aerolineas Argentinas,False,Boeing 737 NG,13.2,Boeing,"[10.0, 15.0)"
1,DAL14_20180101,a14c29,N183DN,B763,KATL,NaN,EDDF,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Delta Air Lines,False,Boeing 767,31.6,Boeing,"[30.0, 35.0)"
2,JBU263_20180108,aa600a,N768JB,A320,KJFK,NaN,KSEA,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,JetBlue Airways,False,Airbus A320,15.9,Airbus,"[15.0, 20.0)"
3,DAL65_20180108,ab2855,N818NW,A333,KATL,KLAX,KLAX,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Delta Air Lines,False,Airbus A330,17.4,Airbus,"[15.0, 20.0)"
4,EDW24_20180111,4b1901,HB-JMF,A343,LSZH,LSZH,MMUN,LSZH,engine,return,...,4b382175,engine,return,unknown,Edelweiss Air,False,Airbus A340,21.1,Airbus,"[20.0, 25.0)"


In [173]:
age_category_counts = df['age_categories'].value_counts().sort_index().reset_index()
age_category_counts['age_categories_str'] = age_category_counts['age_categories'].astype(str)
age_category_counts

,age_categories,count,age_categories_str
0,"[0.0, 5.0)",0,"[0.0, 5.0)"
1,"[5.0, 10.0)",155,"[5.0, 10.0)"
2,"[10.0, 15.0)",121,"[10.0, 15.0)"
3,"[15.0, 20.0)",115,"[15.0, 20.0)"
4,"[20.0, 25.0)",132,"[20.0, 25.0)"
5,"[25.0, 30.0)",67,"[25.0, 30.0)"
6,"[30.0, 35.0)",37,"[30.0, 35.0)"
7,"[35.0, 40.0)",6,"[35.0, 40.0)"
8,"[40.0, 45.0)",0,"[40.0, 45.0)"
9,"[45.0, 50.0)",2,"[45.0, 50.0)"


In [174]:
age_category_counts['age_categories'][0]

Interval(0.0, 5.0, closed='left')

##### Plot

In [175]:
fig = go.Figure(data=[
    go.Bar(x=age_category_counts['age_categories_str'], y=age_category_counts['count'], name="Age", marker_color="red")
])

fig.update_layout(
    title={'text': "Number of IFE Aircrafts per Age Category", 'font': {"size": 24}},
    xaxis_title="Age Category (Years)",
    yaxis_title="IFE Aircraft Frequency"
)

## Frequency per type BAR

##### Data prep

In [176]:
type_name_frequency = df.groupby('productionLine').flight_id.count().sort_values(ascending=False)
type_name_frequency.index

Index(['Boeing 737 NG', 'Airbus A320', 'Airbus A319', 'Boeing 777',
       'Boeing 767', 'Airbus A321', 'Embraer ERJ170', 'Airbus A330',
       'Boeing 757', 'Boeing 787', 'Boeing 747', 'Canadair CRJ 900',
       'Airbus A300', 'Canadair CRJ 100', 'Bombardier DHC-8',
       'McDonnell Douglas MD-11', 'Embraer ERJ145', 'Airbus A380',
       'Boeing 737 Classic', 'Sukhoi Superjet 100', 'Airbus A220',
       'Boeing 717', 'Airbus A340', 'Airbus A350', 'McDonnell Douglas MD-90',
       'Embraer ERJ190', 'ATR 72', 'BAe Avro RJ', 'Airubs A318',
       'Embraer ERJ135', 'Boeing 737 Original', 'Canadair CRJ 700',
       'Canadair CRJ', 'Fokker 100', 'Airbus A320 NEO',
       'McDonnell Douglas MD-88', 'Boeing 727', 'Embraer ERJ140'],
      dtype='object', name='productionLine')

In [177]:
df['productionLine']

0       Boeing 737 NG
1          Boeing 767
2         Airbus A320
3         Airbus A330
4         Airbus A340
            ...      
630       Airbus A320
631        Boeing 787
632     Boeing 737 NG
633     Boeing 737 NG
634    Embraer ERJ170
Name: productionLine, Length: 635, dtype: object

##### Plot

In [205]:
fig2 = go.Figure(data=[
    go.Bar(x=type_name_frequency.index, y=type_name_frequency.values/type_name_frequency.values.sum(), name="Type", marker_color="green")
])

fig2.update_layout(
    title={'text': "Most Frequent IFE Aircrafts", 'font': {"size": 24}},
    xaxis_title="Aircraft Type",
    yaxis_title="IFE Aircraft Rel Freq",
)

## Age + Frequency SCATTER

##### Data Prep

In [179]:
type_name_frequency_df = type_name_frequency.reset_index()
type_name_frequency_df.rename(columns={'flight_id':'frequency'},inplace=True)
type_name_frequency_df

,productionLine,frequency
0,Boeing 737 NG,157
1,Airbus A320,99
2,Airbus A319,44
3,Boeing 777,36
4,Boeing 767,36
5,Airbus A321,35
6,Embraer ERJ170,33
7,Airbus A330,32
8,Boeing 757,25
9,Boeing 787,25


In [180]:
median_age = df.groupby('productionLine').ageYears.median()
median_age.dropna(inplace=True)
median_age_df = median_age.reset_index()
median_age_df.rename(columns={'ageYears':'ageYears_median'}, inplace=True)
median_age_df

,productionLine,ageYears_median
0,ATR 72,9.65
1,Airbus A220,6.80
2,Airbus A300,29.55
3,Airbus A319,19.65
4,Airbus A320,14.50
5,Airbus A320 NEO,5.80
6,Airbus A321,9.40
7,Airbus A330,16.95
8,Airbus A340,21.10
9,Airbus A350,9.45


In [181]:
freq_medianage_merged = type_name_frequency_df.merge(median_age_df, on='productionLine')
freq_medianage_merged

,productionLine,frequency,ageYears_median
0,Boeing 737 NG,157,15.30
1,Airbus A320,99,14.50
2,Airbus A319,44,19.65
3,Boeing 777,36,19.25
4,Boeing 767,36,25.85
5,Airbus A321,35,9.40
6,Embraer ERJ170,33,9.30
7,Airbus A330,32,16.95
8,Boeing 757,25,26.80
9,Boeing 787,25,9.10


##### Plot

In [182]:
fig3 = go.Figure(data=[
    go.Scatter(x=freq_medianage_merged['ageYears_median'], y=freq_medianage_merged['frequency'], name="Type",mode='markers',hovertext=freq_medianage_merged['productionLine'], marker_color="blue")
])

fig3.update_layout(
    title={'text': "Age and Frequency per Aircraft Type", 'font': {"size": 24}},
    xaxis_title="Median Age (Years)",
    yaxis_title="IFE Aircraft Frequency",
)

In [183]:
production_dict = {'ATR 72' : '',
                   'Airbus A220' :'', 
                   'Airbus A300' :'',
                   'Airbus A319' :'',
                   'Airbus A320' :'',
                   'Airbus A320 NEO' :'',
                   'Airbus A321' :'',
                   'Airbus A330' :'',
                   'Airbus A340' :'',
                   'Airbus A350' :'',
                   'Airbus A380' :'',
                   'Airubs A318' :'',
                   'BAe Avro RJ' :'',
                   'Boeing 717' :'',
                   'Boeing 727' :'',
                   'Boeing 737' :'',
                   'Boeing 737 Classic' :'', 
                   'Boeing 737 NG' :'',
                   'Boeing 737 Original' :'',
                   'Boeing 747' :'',
                   'Boeing 757' :'',
                   'Boeing 767' :'',
                   'Boeing 777' :'',
                   'Boeing 787' :'',
                   'Bombardier DHC-8' :'',
                   'Canadair CRJ' :'',
                   'Canadair CRJ 100' :'',
                   'Canadair CRJ 700' :'',
                   'Canadair CRJ 900' :'',
                   'Embraer ERJ135' :'',
                   'Embraer ERJ140' :'',
                   'Embraer ERJ145' :'',
                   'Embraer ERJ170' :'',
                   'Embraer ERJ190' :'',
                   'Fokker 100' :'',
                   'McDonnell Douglas MD-11' :'',
                   'McDonnell Douglas MD-88' :'',
                   'McDonnell Douglas MD-90' :119,
                   'Sukhoi Superjet 100':229
 }

In [184]:
diverted_df = df[(~df['landing'].isna()) & (df['landing'] != df['destination']) & (df['origin'] != df['destination'])]
diverted_df

,flight_id,icao24,registration,typecode,origin,landing,destination,diverted,tweet_problem,tweet_result,...,avh_id,avh_problem,avh_result,avh_fueldump,airlineName,isFreighter,productionLine,ageYears,manufacturer,age_categories
4,EDW24_20180111,4b1901,HB-JMF,A343,LSZH,LSZH,MMUN,LSZH,engine,return,...,4b382175,engine,return,unknown,Edelweiss Air,False,Airbus A340,21.1,Airbus,"[20.0, 25.0)"
12,CPZ6043_20180129,a1ac11,N207AN,E75L,KLAX,KELP,KSAN,KELP,NaN,NaN,...,NaN,NaN,NaN,NaN,Envoy Air,False,Embraer ERJ170,9.4,Embraer,"[5.0, 10.0)"
14,BLX236_20180201,4ac8d9,SE-RFY,B738,GCLP,EDDN,EFHK,EDDN,unclear,diverted,...,NaN,NaN,NaN,NaN,TUIfly Nordic,False,Boeing 737 NG,10.7,Boeing,"[10.0, 15.0)"
16,AFR032_20180209,3949e1,F-GSPB,B772,LFPO,LFPO,KJFK,LFPO,technical,return,...,4b4b9911,engine,return,unknown,Air France,False,Boeing 777,26.6,Boeing,"[25.0, 30.0)"
18,BOS3_20180213,3982a8,F-HAVI,B752,LFPO,LFPO,KEWR,LFPO,unclear,return,...,NaN,NaN,NaN,NaN,Air France,False,Boeing 757,34.3,Boeing,"[30.0, 35.0)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
619,ASH6028_20200115,abdd1c,N86350,E75L,KAUS,KIAH,KIAD,KIAH,unclear,diverted,...,NaN,NaN,NaN,NaN,Mesa Airlines,False,Embraer ERJ170,7.5,Embraer,"[5.0, 10.0)"
621,ENY4130_20200119,a91e11,N687JS,E145,KORD,KORD,KLSE,KORD,landing_gear,return,...,NaN,NaN,NaN,NaN,Envoy Air,False,Embraer ERJ145,20.2,Embraer,"[20.0, 25.0)"
622,AAL2618_20200119,aa15cc,N749US,A319,KDFW,KXNA,KRNO,KXNA,NaN,NaN,...,NaN,NaN,NaN,NaN,American Airlines,False,Airbus A319,24.2,Airbus,"[20.0, 25.0)"
623,QXE2712_20200120,a822d6,N623QX,E75L,KSEA,KBOI,KDAL,KBOI,smoke_burn_smell_flames,diverted,...,4d240887,smoke_burn_smell_flames,diverted,unknown,Horizon Air,False,Embraer ERJ170,7.6,Embraer,"[5.0, 10.0)"


In [208]:
df['diverted'] = (~df['landing'].isna()) & (df['landing'] != df['destination']) & (df['origin'] != df['destination'])
df

,flight_id,icao24,registration,typecode,origin,landing,destination,diverted,tweet_problem,tweet_result,...,avh_id,avh_problem,avh_result,avh_fueldump,airlineName,isFreighter,productionLine,ageYears,manufacturer,age_categories
0,ARG1511_20180101,e06442,LV-FQB,B738,SACO,SABE,SABE,False,NaN,NaN,...,NaN,NaN,NaN,NaN,Aerolineas Argentinas,False,Boeing 737 NG,13.2,Boeing,"[10.0, 15.0)"
1,DAL14_20180101,a14c29,N183DN,B763,KATL,NaN,EDDF,False,NaN,NaN,...,NaN,NaN,NaN,NaN,Delta Air Lines,False,Boeing 767,31.6,Boeing,"[30.0, 35.0)"
2,JBU263_20180108,aa600a,N768JB,A320,KJFK,NaN,KSEA,False,NaN,NaN,...,NaN,NaN,NaN,NaN,JetBlue Airways,False,Airbus A320,15.9,Airbus,"[15.0, 20.0)"
3,DAL65_20180108,ab2855,N818NW,A333,KATL,KLAX,KLAX,False,NaN,NaN,...,NaN,NaN,NaN,NaN,Delta Air Lines,False,Airbus A330,17.4,Airbus,"[15.0, 20.0)"
4,EDW24_20180111,4b1901,HB-JMF,A343,LSZH,LSZH,MMUN,True,engine,return,...,4b382175,engine,return,unknown,Edelweiss Air,False,Airbus A340,21.1,Airbus,"[20.0, 25.0)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
630,BAW12DT_20200127,400a26,G-EUUR,A320,LHBP,NaN,EGLL,False,unclear,planned,...,NaN,NaN,NaN,NaN,British Airways,False,Airbus A320,21.5,Airbus,"[20.0, 25.0)"
631,AMX057_20200128,ad7633,N967AM,B788,RJAA,NaN,MMMX,False,NaN,NaN,...,NaN,NaN,NaN,NaN,Aeromexico,False,Boeing 787,10.7,Boeing,"[10.0, 15.0)"
632,DAL392_20200128,a45f59,N381DN,B738,MPTO,KATL,KATL,False,NaN,NaN,...,NaN,NaN,NaN,NaN,Delta Air Lines,False,Boeing 737 NG,25.2,Boeing,"[25.0, 30.0)"
633,SWA2192_20200129,abc478,N8575Z,B738,KDEN,KHOU,KHOU,False,NaN,NaN,...,NaN,NaN,NaN,NaN,Southwest Airlines,False,Boeing 737 NG,6.5,Boeing,"[5.0, 10.0)"


In [189]:
diverted_type_df = df[df['diverted']==True].groupby('productionLine').flight_id.count().reset_index()
diverted_type_df.rename(columns={'flight_id':'diversions'}, inplace=True)
diverted_type_df.sort_values(by='diversions',ascending=False, inplace=True)
diverted_type_df

,productionLine,diversions
11,Boeing 737 NG,43
3,Airbus A320,31
2,Airbus A319,18
14,Boeing 767,16
15,Boeing 777,12
22,Embraer ERJ170,11
13,Boeing 757,9
16,Boeing 787,8
4,Airbus A321,8
5,Airbus A330,8


In [200]:
fig4 = go.Figure(data=[
    go.Bar(x=diverted_type_df['productionLine'], y=diverted_type_df['diversions']/diverted_type_df['diversions'].sum(), name="Type",hovertext=diverted_type_df['productionLine'], marker_color="blue")
])

fig4.update_layout(
    title={'text': "Normalized Diversions per Aircraft Type", 'font': {"size": 24}},
    xaxis_title="Aircraft Type",
    yaxis_title="Diversions",
)

In [193]:
grouped_by_type_df = df.groupby('productionLine').flight_id.count().sort_values(ascending=False).reset_index()
grouped_by_type_df

,productionLine,flight_id
0,Boeing 737 NG,157
1,Airbus A320,99
2,Airbus A319,44
3,Boeing 777,36
4,Boeing 767,36
5,Airbus A321,35
6,Embraer ERJ170,33
7,Airbus A330,32
8,Boeing 757,25
9,Boeing 787,25


,productionLine,diversions
11,Boeing 737 NG,43
3,Airbus A320,31
2,Airbus A319,18
14,Boeing 767,16
15,Boeing 777,12
22,Embraer ERJ170,11
13,Boeing 757,9
16,Boeing 787,8
4,Airbus A321,8
5,Airbus A330,8


In [197]:
merged_diversion_types = grouped_by_type_df.merge(diverted_type_df)
merged_diversion_types

,productionLine,flight_id,diversions
0,Boeing 737 NG,157,43
1,Airbus A320,99,31
2,Airbus A319,44,18
3,Boeing 777,36,12
4,Boeing 767,36,16
5,Airbus A321,35,8
6,Embraer ERJ170,33,11
7,Airbus A330,32,8
8,Boeing 757,25,9
9,Boeing 787,25,8


In [199]:
global_rate = merged_diversion_types['diversions'].sum() / merged_diversion_types['flight_id'].sum()

# Use the average number of emergencies as the weighting factor (k)
k = merged_diversion_types['flight_id'].mean()

# Calculate the adjusted diversion rate
merged_diversion_types['adjusted_diversion_rate'] = (
    (merged_diversion_types['diversions'] + k * global_rate) /
    (merged_diversion_types['flight_id'] + k)
)

# Sort by the adjusted diversion rate
merged_diversion_types_sorted = merged_diversion_types.sort_values('adjusted_diversion_rate', ascending=False)
merged_diversion_types_sorted

,productionLine,flight_id,diversions,adjusted_diversion_rate
18,Boeing 737 Classic,5,5,0.447297
12,Airbus A300,10,6,0.411561
19,Sukhoi Superjet 100,3,3,0.407246
14,Bombardier DHC-8,8,5,0.406135
4,Boeing 767,36,16,0.400000
2,Airbus A319,44,18,0.382507
17,Airbus A380,5,3,0.379730
20,Airbus A340,3,2,0.371014
23,Embraer ERJ135,1,1,0.360937
24,Boeing 727,1,1,0.360937


In [ ]:
fig5 = go.Figure(data=[
    go.Bar(x=diverted_type_df['productionLine'], y=diverted_type_df['diversions'], name="Type",hovertext=diverted_type_df['productionLine'], marker_color="blue")
])

fig5.update_layout(
    title={'text': "Diversions per Aircraft Type", 'font': {"size": 24}},
    xaxis_title="Aircraft Type",
    yaxis_title="Diversions",
)

In [207]:
with open('../data/processed/production_dict.json', 'r') as reg_file:
    prod_dict = json.load(reg_file)
df_production = pd.DataFrame(list(prod_dict.items()), columns=['model_family', 'total_produced'])

# Normalize and sort
df_production['total_produced_norm'] = df_production.apply(lambda row : row['total_produced']/df_production['total_produced'].sum(), axis=1)
df_production.sort_values(by='total_produced_norm', ascending=False,inplace=True)

# Plot
prod_fig = go.Figure(data=[
    go.Bar(x=df_production['model_family'], y=df_production['total_produced_norm'], name="model_family", marker_color="blue")
])

prod_fig.update_layout(
    title={'text': "Normalized Amount of Aircraft per Aircraft Model Family", 'font': {"size": 24}},
    xaxis_title="Aircraft Model Family",
    yaxis_title="Normalized Frequency"
)

In [256]:
fig_squawk_prod = go.Figure(data=[
    go.Bar(x=type_name_frequency.index, y=type_name_frequency.values/type_name_frequency.values.sum(), name="Emergencies (norm)", marker_color="green"),
    go.Bar(x=diverted_type_df['productionLine'], y=diverted_type_df['diversions']/diverted_type_df['diversions'].sum(), name="Diversions (norm)",hovertext=diverted_type_df['productionLine'], marker_color="red"),
    go.Bar(x=df_production['model_family'], y=df_production['total_produced_norm'], name="Production (norm)", marker_color="blue"),
])

fig_squawk_prod.update_layout(
    title={'text': "In-Flight Emergencies vs Diversions vs Production", 'font': {"size": 24}},
    xaxis_title="Aircraft Model Family",
    yaxis_title="Normalized Frequency",
    width=1400, 
    height=800,
    paper_bgcolor='rgb(254, 246, 224)'

)

fig_squawk_prod.update_xaxes(tickangle=60)

In [216]:
fig_div_prod = go.Figure(data=[
    go.Bar(x=df_production['model_family'], y=df_production['total_produced_norm'], name="Production (norm)", marker_color="blue"),
    go.Bar(x=diverted_type_df['productionLine'], y=diverted_type_df['diversions']/diverted_type_df['diversions'].sum(), name="Diversions (norm)",hovertext=diverted_type_df['productionLine'], marker_color="red")
])

fig_div_prod.update_layout(
    title={'text': "Production vs In-Flight Emergencies", 'font': {"size": 24}},
    xaxis_title="Aircraft Model Family",
    yaxis_title="Normalized Frequency"
)

In [232]:
avg_age_type = df.groupby('productionLine').ageYears.mean().round(decimals=1)

In [251]:
fig_age_squawk_prod = go.Figure(data=[
    go.Bar(x=type_name_frequency.index, y=type_name_frequency.values/type_name_frequency.values.sum(), name="Emergencies (norm)", marker_color="green"),
    go.Bar(x=df_production['model_family'], y=df_production['total_produced_norm'], name="Production (norm)", marker_color="blue"),
    go.Bar(x=avg_age_type.index, y=avg_age_type.values/avg_age_type.values.sum(), name="Age (norm)",hovertext=diverted_type_df['productionLine'], marker_color='#ff7f0e')
])

fig_age_squawk_prod.update_layout(
    width=1400, 
    height=800,
    title={'text': "SQUAWK - Production - Age", 'font': {"size": 24}},
    xaxis_title="Aircraft Model Family",
    yaxis_title="Normalized Frequency"
)

fig_age_squawk_prod.update_xaxes(tickangle=60)